In [1]:
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.3.0 (SDL 2.24.2, Python 3.9.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
distance_to_junc = 20
direction_angle = 20
radius = 100

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=24, type=spectator), Actor(id=25, type=traffic.stop), Actor(id=26, type=traffic.stop), Actor(id=27, type=traffic.stop), Actor(id=28, type=traffic.stop), Actor(id=29, type=traffic.stop), Actor(id=30, type=traffic.stop), Actor(id=31, type=traffic.stop), Actor(id=32, type=traffic.stop), Actor(id=33, type=traffic.yield), Actor(id=34, type=traffic.yield), Actor(id=35, type=traffic.yield), Actor(id=36, type=traffic.unknown), Actor(id=37, type=traffic.unknown), Actor(id=38, type=traffic.yield), Actor(id=39, type=traffic.unknown), Actor(id=40, type=traffic.unknown), Actor(id=41, type=traffic.speed_limit.40), Actor(id=42, type=traffic.speed_limit.40), Actor(id=43, type=traffic.speed_limit.90), Actor(id=44, type=traffic.speed_limit.90), Actor(id=45, type=traffic.speed_limit.90), Actor(id=46, type=traffic.speed_limit.90), Actor(id=47, type=traffic.speed_limit.90), Actor(id=48, type=traffic.speed_limit.90), Actor(id=49, type=traffic.speed_limit.90), Actor(id=50, type=traffic.speed_limit.

In [3]:
manual_car = world.get_actor(147)
ego_vehicle = world.get_actor(147)
#second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=-32.236683, y=6.291035, z=10.441302)


In [5]:
def get_wp_x_units_to_left(ego_vehicle, x):
    location = ego_vehicle.get_location()
    transform = ego_vehicle.get_transform()

    ego_forward_vector = transform.get_forward_vector()
    left_vector = carla.Vector3D(x=-ego_forward_vector.y, y=ego_forward_vector.x, z=ego_forward_vector.z)

    target_location = location - left_vector * 3
    target_location = carla.Location(target_location.x, target_location.y, target_location.z)
    return target_location
    wp = world_map.get_waypoint(target_location)
    print("Old location: ", location)
    print("Target location: ", target_location)

In [16]:
location = ego_vehicle.get_location()
transform = ego_vehicle.get_transform()
ego_forward_vector_1 = transform.get_forward_vector()

In [17]:
print(ego_forward_vector)

Vector3D(x=-0.623098, y=0.782144, z=0.000551)


In [19]:
def angle_between_vectors(a, b):
    dot = a.x * b.x + a.y * b.y
    mag_a = math.sqrt(a.x**2 + a.y**2)
    mag_b = math.sqrt(b.x**2 + b.y**2)
    cos_theta = dot / (mag_a * mag_b)
    theta = math.acos(cos_theta)
    return math.degrees(theta)

In [21]:
angle_between_vectors(ego_forward_vector_1, ego_forward_vector_1)

0.0

In [6]:
def check_ego_on_highway(ego_vehicle_location, road_lane_ids, world_map):
    waypoints = []
    ego_waypoint = world_map.get_waypoint(ego_vehicle_location)
    waypoints.append(ego_waypoint)
    if ego_waypoint.get_left_lane() is not None:
        waypoints.append(ego_waypoint.get_left_lane())
    if ego_waypoint.get_right_lane() is not None:
        waypoints.append(ego_waypoint.get_right_lane())
    for wp in waypoints:
        ego_vehilce_road_id = wp.road_id
        lanes = []
        for id in road_lane_ids:
            if str(ego_vehilce_road_id) == id.split("_")[0]:
                lanes.append(id.split("_")[1])
        lanes = [int(lane) for lane in lanes]
        if len(lanes) >= 6 or (
            sorted(lanes) == list(range(min(lanes), max(lanes) + 1)) and len(lanes) >= 3
        ):
            return True

    return False


In [7]:
def create_city_matrix(ego_vehicle_location, road_lane_ids, world_map, ghost=False, ego_on_bad_highway_street = False):
    """
    Create a matrix representing the lanes around the ego vehicle.

    Parameters:
        ego_vehicle (Vehicle): The ego vehicle for which to create the city matrix.
        road_lane_ids (list of str): List of road and lane IDs in the world map.
        world_map (WorldMap): The map representing the environment.

    Returns:
        collections.OrderedDict: An ordered dictionary representing the city matrix.
                                 The keys are lane IDs in the format "road_id_lane_id".
                                 The values are lists of integers indicating lane directions:
                                     0 - Lane in the same direction as ego vehicle's lane.
                                     1 - Ego vehicle's lane.
                                     3 - Empty lane (no traffic).
    """
    # Get lane id for ego_vehicle
    ego_vehilce_waypoint = world_map.get_waypoint(ego_vehicle_location)
    ego_vehilce_lane_id = str(ego_vehilce_waypoint.lane_id)
    print("ego_vehicle_lane_id: ", ego_vehilce_lane_id)
    ego_vehilce_road_id = str(ego_vehilce_waypoint.road_id)

    lanes = []
    for id in road_lane_ids:
        if ego_vehilce_road_id == id.split("_")[0]:
            lanes.append(id.split("_")[1])
    lanes.sort()
    lanes = [int(id) for id in lanes]
    lanes_splitted = []
    z = 0
    for i in range(1, len(lanes)):
        if lanes[i] == lanes[i - 1] - 1 or lanes[i] == lanes[i - 1] + 1:
            continue
        else:
            lanes_splitted.append(lanes[z:i])
            z = i
    lanes_splitted.append(lanes[z:])
    lanes_splitted = [
        sorted(direction, key=abs, reverse=True) for direction in lanes_splitted
    ]

    # Initialize matrix and key_value_pairs
    matrix = None
    key_value_pairs = None
    other_direction = []
    ego_direction = []
    for direction in lanes_splitted:
        if int(ego_vehilce_lane_id) in direction:
            ego_direction = direction
        else:
            other_direction = direction
    if len(ego_direction) >= 4:
        key_value_pairs = [
            ("left_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("left_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[3]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[2]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[1]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[0]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            ("right_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
        ]
    elif len(ego_direction) == 3:
        key_value_pairs = [
            ("left_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("left_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[2]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[1]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[0]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            ("No_4th_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
        ]

    elif len(ego_direction) == 2 and len(other_direction) == 2:
        key_value_pairs = [
            ("left_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("left_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            (
                ego_vehilce_road_id + "_" + str(other_direction[0]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(other_direction[1]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[1]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[0]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            ("right_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
        ]
    elif len(ego_direction) == 2 and len(other_direction) == 0:
        key_value_pairs = [
            ("left_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("left_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("No_opposing_direction", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("No_opposing_direction", [3, 3, 3, 3, 3, 3, 3, 3]),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[1]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[0]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            ("right_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
        ]
    elif len(ego_direction) == 1 and len(other_direction) == 1:
        key_value_pairs = [
            ("left_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("left_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("No_opposing_direction", [3, 3, 3, 3, 3, 3, 3, 3]),
            (
                ego_vehilce_road_id + "_" + str(other_direction[0]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[0]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            ("No_own_right_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
        ]
    elif len(ego_direction) == 1 and len(other_direction) == 0:
        key_value_pairs = [
            ("left_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("left_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("No_other_right_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("No_opposing_direction", [3, 3, 3, 3, 3, 3, 3, 3]),
            (
                ego_vehilce_road_id + "_" + str(ego_direction[0]),
                [0, 0, 0, 0, 0, 0, 0, 0],
            ),
            ("No_own_right_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_inner_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
            ("right_outer_lane", [3, 3, 3, 3, 3, 3, 3, 3]),
        ]

    # Update matrix
    if key_value_pairs:
        matrix = collections.OrderedDict(key_value_pairs)
    if matrix and not ghost:
        try:
            if ego_on_bad_highway_street:
                if int(ego_vehilce_lane_id) > 0:
                    matrix[str(ego_vehilce_road_id) + "_" + str(int(ego_vehilce_lane_id) + 1)][3] = 1
                else:
                    matrix[str(ego_vehilce_road_id) + "_" + str(int(ego_vehilce_lane_id) - 1)][3] = 1
            else:
                matrix[str(ego_vehilce_road_id) + "_" + str(ego_vehilce_lane_id)][3] = 1
        except KeyError:
            matrix[str(ego_vehilce_road_id) + "_" + str(ego_vehilce_lane_id)][3] = 1
    return matrix

In [73]:
def update_matrix(
    world_map,
    ego_vehicle,
    ego_location,
    highway_shape,
    wps,
    matrix,
    ego_waypoint,
    junction,
    cars_on_entryExit,
    direction_angle,
    ghost=False,
    on_entry = False,
):
    if highway_shape[1] == 3:
        matrix[list(matrix.keys())[5]] = [0, 0, 0, 0, 0, 0, 0, 0]

    matrix = list(matrix.items())
    exit_entry_found = False
    entry_wps, exit_wps = highway_shape[2], highway_shape[3]

    # TODO: put in a config file
    # set the angle threshold depending on curve type
    if highway_shape[0] == "entry_and_exit":
        degree = 30
    elif highway_shape[0] in ["single_entry", "single_exit"]:
        degree = 20
    elif highway_shape[0] in ["dual_entry"] or highway_shape[0] in ["dual_exit"]:
        degree = 18  # TODO: test and fine tuning of degree
    else:
        # just in case
        degree = 20

    # get right_lane_end_wp & right_lane_start_wp for dual entry & exit surrounding cars angle calculation
    for wps_group in wps:
        if (len(wps_group) == 4) and (wps_group[0][1] == "end"):
            right_lane_end_wp = get_right_lane_wp(wps_group)
        elif (len(wps_group) == 4) and (wps_group[0][1] == "start"):
            right_lane_start_wp = get_right_lane_wp(wps_group)

    # First: Look behind ego
    exit_entry_found, col_entryExit, matrix = search_entry_or_exit(
        world_map,
        ego_location,
        "behind",
        exit_entry_found,
        matrix,
        highway_shape,
        wps,
        entry_wps,
        exit_wps,
        degree,
        junction.id,
        right_lane_end_wp,
        right_lane_start_wp,
    )
    # if not yet found, look in front
    if exit_entry_found == False:
        _, col_entryExit, matrix = search_entry_or_exit(
            world_map,
            ego_location,
            "before",
            exit_entry_found,
            matrix,
            highway_shape,
            wps,
            entry_wps,
            exit_wps,
            degree,
            junction.id,
            right_lane_end_wp,
            right_lane_start_wp,
        )

    # get road ids of: entry/exit, city road before and highway road after junction object
    entry_road_id = []
    exit_road_id = []
    entry_city_road = []
    exit_city_road = []
    entry_city_lane_id = []    
    exit_city_lane_id = []
    entry_highway_road = []
    exit_highway_road = []
    entry_highway_lane_id = []
    exit_highway_lane_id = []

    if entry_wps:
        for entry_wp in entry_wps[0]:
            entry_city_road.append(entry_wp.previous(3)[0].road_id)
            entry_city_lane_id.append(abs(entry_wp.previous(3)[0].lane_id))
            entry_road_id.append(entry_wp.road_id)
        for entry_wp in entry_wps[1]:
            entry_highway_road.append(entry_wp.next(3)[0].road_id)
            entry_highway_lane_id.append(abs(entry_wp.next(3)[0].lane_id))
    if exit_wps:
        for exit_wp in exit_wps[1]:
            exit_city_road.append(exit_wp.next(3)[0].road_id)
            exit_city_lane_id.append(abs(exit_wp.next(3)[0].road_id))
            exit_road_id.append(exit_wp.road_id)
        for exit_wp in exit_wps[0]:
            exit_highway_road.append(exit_wp.previous(3)[0].road_id)
            exit_highway_lane_id.append(abs(exit_wp.previous(3)[0].lane_id))


    # detect surrounding cars on exit/entry
    for car in cars_on_entryExit:
        col = calculate_position_in_matrix(
            ego_location,
            ego_vehicle,
            car,
            dict(matrix),
            world_map,
            ego_vehicle.get_velocity(),
            ghost,
        )
        if col is None:
            continue
        # matrix = list(matrix)
        if matrix:
            ego_already_in_matrix = False
            for key, row in matrix:
                if 1 in row:
                    ego_already_in_matrix = True
            print("Ego already in matrix: ", ego_already_in_matrix)
            if (ego_already_in_matrix and car.id == ego_vehicle.id):
                continue
            other_car_waypoint = world_map.get_waypoint(car.get_location())
            other_car_road_id = other_car_waypoint.road_id
            other_car_lane_id = other_car_waypoint.lane_id
            
            if other_car_road_id in entry_city_road + exit_city_road and not on_entry:
                continue
            distance_clostest_starting_waypoint = distance(get_clostest_starting_waypoint(junction.get_waypoints(carla.LaneType().Driving), car.get_location()).transform.location, car.get_location())
            if other_car_road_id in entry_city_road + exit_city_road:
                if distance_clostest_starting_waypoint > 30:
                    continue
                elif distance_clostest_starting_waypoint < 15:
                    row = 6
                else:
                    row = 7
                if highway_shape[0] in ["entry_and_exit"]:
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                elif highway_shape[0] in ["dual_entry"]:
                    entry_city_lane_id.sort()
                    # dual entry
                    if abs(other_car_lane_id) == entry_city_lane_id[0]:
                        col_entry = col_entryExit
                    elif abs(other_car_lane_id) == entry_city_lane_id[1]:
                        col_entry = col_entryExit + 1


                elif highway_shape[0] in ["dual_exit"]:
                    entry_city_lane_id.sort()
                    # dual exit
                    if abs(other_car_lane_id) == entry_city_lane_id[0]:
                        col_entry = col_entryExit +1
                    elif abs(other_car_lane_id) == entry_city_lane_id[1]:
                        col_entry = col_entryExit
                else:
                    col_entry = col_entryExit
                    col_exit = col_entryExit
            


                if entry_city_road == exit_city_road:
                    if is_junction_behind(other_car_waypoint, 40):
                        check_junction = get_junction_behind(other_car_waypoint, 40)
                        road_lane_ids = get_all_road_lane_ids(world_map)
                        if is_highway_junction(car, other_car_waypoint, check_junction, road_lane_ids, direction_angle):
                            on_exit_street = True
                        else:
                            on_exit_street = False
                else:
                    on_exit_street = False
                if (other_car_road_id in entry_city_road) and (col_entry < 8) and not on_exit_street:
                    insert_in_matrix(matrix, car, ego_vehicle, col_entry, row)
                elif (other_car_road_id in exit_city_road) and (col_exit < 8) and on_exit_street:
                    insert_in_matrix(matrix, car, ego_vehicle, col_exit, row)
                continue
                
            distance_junction_end = get_distance_junction_end(
                other_car_waypoint
            )
            distance_junction_start = get_distance_junction_start(
                other_car_waypoint
            )
            if other_car_road_id in entry_road_id + entry_highway_road:
                if highway_shape[0] == "dual_entry": 
                    yaw_difference = abs(
                        right_lane_end_wp.transform.rotation.yaw
                    ) - abs(
                        right_lane_end_wp.previous(distance_junction_end)[
                            0
                        ].transform.rotation.yaw
                    )
                else:
                    yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(
                        entry_wps[1][0]
                        .previous(distance_junction_end)[0]
                        .transform.rotation.yaw
                    )
            elif other_car_road_id in exit_road_id + exit_highway_road:
                if highway_shape[0] == "dual_exit":
                    yaw_difference = abs(
                        right_lane_start_wp.transform.rotation.yaw
                    ) - abs(
                        right_lane_start_wp.next(distance_junction_start)[
                            0
                        ].transform.rotation.yaw
                    )
                else:

                    yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(
                        exit_wps[0][0]
                        .next(distance_junction_start)[0]
                        .transform.rotation.yaw
                    )
            try:
                already_in_curve = abs(yaw_difference) > degree
                already_after_curve = abs(yaw_difference) > degree * 2
            except UnboundLocalError:
                already_after_curve = False
                already_in_curve = False
            if not on_entry:
                if already_after_curve:
                    row = 7
                elif already_in_curve:

                    row = 6
                else:
                    row = 5
            else:
                row = 5
                """
                distance_bounding_box = distance(junction.bounding_box.location, car.get_location())
                print(distance_bounding_box)
                if distance_bounding_box < 30:
                    row = 5
                if distance_bounding_box < 40:
                    row = 6
                else:
                    row = 7
                """
            if already_in_curve:
                if highway_shape[0] in ["entry_and_exit"]:
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                elif highway_shape[0] in ["dual_entry"]:
                    # dual entry
                    entry_road_id = [
                        x for _, x in sorted(zip(entry_highway_lane_id, entry_road_id))
                    ]
                    entry_highway_lane_id.sort()
                    if other_car_road_id == entry_road_id[0]:
                        col_entry = col_entryExit
                    elif other_car_road_id == entry_road_id[1]:
                        col_entry = col_entryExit + 1
                elif highway_shape[0] in ["dual_exit"]:
                    # dual exit
                    exit_road_id = [
                        x for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                    ]
                    exit_highway_lane_id.sort()
                    if other_car_road_id == exit_road_id[0]:
                        col_exit = col_entryExit + 1
                    elif other_car_road_id == exit_road_id[1]:
                        col_exit = col_entryExit  # + 1
                else:
                    col_entry = col_entryExit
                    col_exit = col_entryExit
            else:
                if (highway_shape[1] == 3) and (col <= (col_entryExit + 1)):
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                else:
                    if highway_shape[0] in ["dual_entry"]:
                        # dual entry
                        entry_road_id = [
                            x
                            for _, x in sorted(
                                zip(entry_highway_lane_id, entry_road_id)
                            )
                        ]
                        entry_highway_lane_id.sort()
                        if other_car_road_id == entry_road_id[0]:
                            row = row - 1
                    elif highway_shape[0] in ["dual_exit"]:
                        # dual exit
                        exit_road_id = [
                            x
                            for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                        ]
                        exit_highway_lane_id.sort()
                        if other_car_road_id == exit_road_id[0]:
                            row = row - 1
                    col_entry = col
                col_exit = col

            entry_streets = entry_road_id + entry_highway_road
            exit_streets = exit_road_id + exit_highway_road
            if (other_car_road_id in entry_streets) and (col_entry < 8):
                insert_in_matrix(matrix, car, ego_vehicle, col_entry, row)

            elif (other_car_road_id in exit_streets) and (col_exit < 8):
                insert_in_matrix(matrix, car, ego_vehicle, col_exit, row)

    return dict(matrix)

In [9]:
def safe_data(ego_vehicle, matrix, street_type):
    steering_angle = get_steering_angle(ego_vehicle)
    speed = get_speed(ego_vehicle)
    print("============================================================")
    if matrix:
        row_data = get_row(matrix)
        row_data["speed"] = speed
        row_data["steering_angle"] = steering_angle
        row_data["street_type"] = street_type
        # df = df.append(row_data, ignore_index=True)

    # clock.tick_busy_loop(60)
    time.sleep(0.3)
    world.tick()

In [69]:
def check_ego_exit_highway(ego_vehicle, highway_forward_vector, highway_shape):
    if highway_forward_vector and highway_shape != "dual exit":
        if True and angle_between_vectors(ego_vehicle.get_transform().get_forward_vector(), highway_forward_vector) >= 8:
            return True
        else:
            return False
    else: 
        return False

In [74]:
same_junction = False
on_junction = False
junctions_detected = False
junction_old = None
wps_old = None
highway_shape_old = None
highway_shape = None
wrong_shape = None
junction_shape = None
matrix = None
junction_id = None
street_type = None 
road_lane_ids = get_all_road_lane_ids(world_map = world.get_map())
on_entry = False
junction_id_skip = None
ego_on_bad_highway_street = False
highway_forward_vector = None
df = initialize_dataframe()
#try:
while True:
    print("----------------------------------")
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    ego_on_highway = check_ego_on_highway(ego_location, road_lane_ids, world_map)
    if not ego_on_highway:
        left_location = get_wp_x_units_to_left(ego_vehicle, 3)
        ego_on_highway = check_ego_on_highway(left_location, road_lane_ids, world_map)
        if ego_on_highway:
            ego_on_bad_highway_street = True
    else:
        ego_on_bad_highway_street = False
    if ego_on_highway:
        distance_to_junc = 80
    else:
        distance_to_junc = 20
    
    if junctions_detected and not ego_waypoint.is_junction:
        junctions_detected = False 
        same_junction = False
    elif junctions_detected:
        pass
    elif ego_waypoint.is_junction and on_junction == False:
        on_junction = True
    elif on_junction:
        junction_old = junction
        wps_old = wps
        highway_shape_old = highway_shape
        on_junction = False
        junctions_detected = True
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not on_junction:
         #and  != get_junction_ahead(ego_waypoint, distance_to_junc).id:
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_id = junction.id
        highway_junction = is_highway_junction(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle)


    # CASES
    # 1. On city junction
    if ego_waypoint.is_junction and not highway_junction and junction.id != 459:
        street_type = "On junction"
        print("On Junction")
        if junction_shape:
            matrix = detect_surrounding_cars_outside_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, junction, yaw, world
            )
            for key, value in matrix.items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")
    
    # 2. City junction ahead
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not highway_junction and junction.id != 459:
        street_type = "Junction ahead"
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_shape, lanes_all_junction, junction_roads_junction, yaw, = getJunctionShape(
            ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle
        )

        if junction_shape != "Error":
            print("Junction ahead")
            matrix = detect_ego_before_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_waypoint.lane_id,
                ego_waypoint,
                distance_to_junc,
            )
            matrix = detect_surrounding_cars_outside_junction(
                matrix,
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, junction, yaw, world
            )

            for key, value in dict(matrix).items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")


    # 3. Highway entry/exit in front or behind
    elif (is_junction_ahead(ego_waypoint, 80) or (is_junction_behind(ego_waypoint, 40) and street_type == "Highway with entry/exit")) and not on_entry and highway_junction and ego_on_highway and junction_id_skip != junction.id and not check_ego_exit_highway(ego_vehicle, highway_forward_vector, highway_shape):
        street_type = "Highway with entry/exit"
        print("Highway Junction detected")
        if distance(ego_location, junction.bounding_box.location) < 30 and not forward_vector_set:
            print("here")
            highway_forward_vector = ego_vehicle.get_transform().get_forward_vector()
            highway_forward_location = ego_vehicle.get_location()
            forward_vector_set = True
            exit_over = False
        if not same_junction and is_junction_ahead(ego_waypoint, 80):
            forward_vector_set = False
            same_junction = True
            # print("selbst erkennnen")
            junction = get_junction_ahead(ego_waypoint, 80)

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
            wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
            # # print grouped wps
            # for group in wps:
            #     print("######################### NEW GROUP #########################")
            #     for wp in group:
            #         print("-------------------")
            #         print(wp[0], wp[1])
            highway_shape = get_highway_shape(wps, junction)
            if not highway_shape is None:
                print(highway_shape[0], highway_shape[1])
        if (highway_shape is None) and (not junction_old is None):
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old
            wrong_shape = True
        if ego_on_bad_highway_street:
            matrix = create_city_matrix(left_location, road_lane_ids, world_map, False, ego_on_bad_highway_street)
        else:
            matrix = create_city_matrix(ego_location, road_lane_ids, world_map, False, ego_on_bad_highway_street)                
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            if not highway_shape is None and (highway_shape[0] != "normal_highway" or wrong_shape):
                matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, ego_waypoint, junction, cars_on_entryExit, direction_angle) 
                if not junction_old is None and junction_old.id != junction.id:
                    matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape_old, wps_old, matrix, ego_waypoint, junction_old, cars_on_entryExit, direction_angle) 
                
        for key, value in matrix.items():
            print(value)
        print("----------------------------------------------------------------------------------------------------------------------")

    # ego exiting Highway
    elif check_ego_exit_highway(ego_vehicle, highway_forward_vector, highway_shape) and not exit_over:
        print("On highway exit")
        street_type = "On highway exit"
        if highway_forward_vector and distance(highway_forward_location, ego_location) > 60:
            print("reset")
            highway_forward_vector = None
            highway_forward_location = None
        
        junction = get_junction_ahead(ego_waypoint, 1)
        if not junction:
            exit_over = True
            continue
        junction_id_skip = junction.id

        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
        wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
        highway_shape = get_highway_shape(wps, junction)

        on_entry = True
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix  = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, direction_angle, True, False) 
                # if not junction_old is None and junction_old.id != junction.id:
                #     matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True, on_entry) 
                
        for key, value in matrix.items():
            print(value)
        print("----------")


    # 4. Highway entry
    elif is_junction_ahead(ego_waypoint, 20) and highway_junction and street_type in  ["Non highway street", "On junction", "On highway entry"]:

        print("On highway entry")
        street_type = "On highway entry"
        on_entry = True
        junction = get_junction_ahead(ego_waypoint, 20)
        junction_id_skip = junction.id

        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
        wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
        highway_shape = get_highway_shape(wps, junction)

        on_entry = True
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix  = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, direction_angle, True, on_entry) 
                # if not junction_old is None and junction_old.id != junction.id:
                #     matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True, on_entry) 
                
        for key, value in matrix.items():
            print(value)
        print("----------")

    # 5. Normal Road
    else:
        highway_forward_vector = None
        highway_forward_location = None
        on_junction = False
        junction_old = None
        on_entry = False
        print("normal road")
        same_junction = False
        if ego_on_highway:
            street_type = "On highway"
        else:
            street_type = "Non highway street"
        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)
        if matrix:
            matrix, _ = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            for key, value in matrix.items():
                print(value)

    

    safe_data(ego_vehicle, matrix, street_type)

    
"""
except Error:
    pass

finally:
    df.to_csv("x_hours.csv")
    vehicles_list = [ego_vehicle]
    print("\ndestroying %d vehicles" % len(vehicles_list))
    client.apply_batch([carla.command.DestroyActor(x) for x in vehicles_list])

    time.sleep(0.5)
"""

----------------------------------
Highway Junction detected
single_entry 4
ego_vehicle_lane_id:  -4
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 0]
[3, 3, 3, 3, 3, 3, 3, 0]
----------------------------------------------------------------------------------------------------------------------
----------------------------------
Highway Junction detected
ego_vehicle_lane_id:  -4
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 0]
[3, 3, 3, 3, 3, 3, 3, 0]
----------------------------------------------------------------------------------------------------------------------
----------------------------------
Highway Junction detected
ego_vehicle_lane_id:  -4
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0

KeyboardInterrupt: 

In [ ]:
# ToDo
 
# NEU
# -------------------------------------------------------------------------------------
 # 1) Simplify everything   
 # 2) config schreiben
 # 3) Doc strings (später)
 
# JP 
#--------------------------------------------------------------------------------------
 # 1) exits
 # 2) Die ekligen junctions: Tankstelle und Highway Ampel (JP)
        # Einfach 4 Spure ballern
        # simpel wie anderen 
 # -----------------------------------------------------------------------------------


# ICH
#------------------------------------------------------------------------------------
 # 1) Highway Exit